In [14]:
%cd /mnt

%load_ext autoreload
%autoreload 2

import os
import pickle

import numpy as np

from utils import get_dataset_from_file
from run_experiment.gas_experiment import run_gas_experiment
from default_parameters import *

from gluonts.mx.distribution import StudentTOutput, MultivariateGaussianOutput
from gluonts.evaluation import make_evaluation_predictions, Evaluator, MultivariateEvaluator
from gluonts.mx.trainer import Trainer
from gluonts.mx.trainer.callback import TrainingHistory
from sklearn.metrics import mean_absolute_error

import matplotlib.pyplot as plt
import json
from pathlib import Path
from gluonts.dataset.split import split
from gluonts.dataset.common import ListDataset

import mxnet as mx
# from my_models.gluonts_models.univariate.feedforward_linear_means._estimator import (
#     SimpleFeedForwardEstimator as FF_gluonts_univariate_linear,
# )
from my_models.gluonts_models.univariate.probabilistic_forecast.feedforward_gas_means._estimator import (
    SimpleFeedForwardEstimator as FF_gluonts_univariate_gas,
)
from my_models.gluonts_models.feedforward_multivariate_linear_means._estimator import (
    SimpleFeedForwardEstimator as FF_gluonts_multivariate_linear,
)
from my_models.gluonts_models.univariate.point_forecast.feedforward_gas_means._estimator import (
    SimpleFeedForwardEstimator as FF_gluonts_univariate_gas_point,
)
# from my_models.gluonts_models.multivariate_feedforward_gas_means._estimator import (
#     SimpleFeedForwardEstimator as FF_gluonts_multivariate_gas,
# )

from my_models.gluonts_models.univariate.probabilistic_forecast.deepar_gas_means._estimator import (
    DeepAREstimator as DeepAR_gluonts_univariate_gas,
)
from my_models.gluonts_models.univariate.wavenet_gas_means._estimator import (
    WaveNetEstimator as WaveNet_gluonts_gas_means,
)
from my_models.gluonts_models.univariate.probabilistic_forecast.transformer_gas_means._estimator import (
    TransformerEstimator as Transformer_gluonts_gas_means,
)
from my_models.gluonts_models.univariate.point_forecast.transformer_gas_means._estimator import (
    TransformerEstimator as Transformer_gluonts_gas_means_point,
)
from my_models.gluonts_models.univariate.probabilistic_forecast.transformer_test._estimator import (
    TransformerEstimator as Transformer_test,
)
from my_models.gluonts_models.univariate.probabilistic_forecast.seq2seq._mq_dnn_estimator import (
    MQCNNEstimator as MQCNN_gluonts_univariate_gas,
)

from run_experiment.dl_model_experiment import GasHybridBlock
from normalizer import GASNormalizer
from sklearn.linear_model import LinearRegression

import time
import optuna
import argparse

/mnt


/opt/conda/envs/clts2/lib/python3.10/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
/opt/conda/envs/clts2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Checking hybrid forward inputs

In [5]:
MODEL, multivariate, ctx, DATASET_NAME, DATASET_FILE_FOLDER, mean_str, var_str = 'transformer', False, 'gpu', 'fred_md', None, 0, 0
model = MODEL
multivariate = multivariate
ctx = ctx
dataset_name = DATASET_NAME



DATASET_TYPE = "gluonts"  # "synthetic"
DATASET_PARAMS = real_world_data_params  # synthetic_generation_params
DATASET_PARAMS["multivariate"] = multivariate
# DATASET_FILE_FOLDER = None #'tsf_data'

NORMALIZER_NAME = "gas_t_student"  # "gas_simple_gaussian", "gas_complex_gaussian"
NORMALIZER_INITIAL_GUESSES = gas_t_stud_initial_guesses  # gas_{name}_*
NORMALIZER_BOUNDS = gas_t_stud_bounds
NORMALIZER_PARAMS = gas_t_stud_params
NORMALIZER_PARAMS['mean_strength'] = mean_str
NORMALIZER_PARAMS['var_strength'] = var_str


MEAN_LAYER_NAME = "gas"  # TODO: gas
MEAN_LAYER_PARAMS = gas_mean_layer_params

DL_MODEL_LIBRARY = "gluonts"  # "torch"
DL_MODEL_NAME = MODEL  # TODO: "transformer"
if model == 'feedforward':
    DL_MODEL_PARAMS = gluonts_feedforward_params
elif model == 'transformer':
    DL_MODEL_PARAMS = gluonts_transformer_params
elif model == 'deepar':
    DL_MODEL_PARAMS = gluonts_deepar_params
elif model == 'wavenet':
    DL_MODEL_PARAMS = gluonts_wavenet_params
elif model == 'mqcnn':
    DL_MODEL_PARAMS = gluonts_mqcnn_params

N_TRAINING_SAMPLES = 5000
N_TEST_SAMPLES = 1000

ROOT_FOLDER = (
    f"RESULTS_{DATASET_NAME}_{model}_{NORMALIZER_NAME}_{MEAN_LAYER_NAME}_{DL_MODEL_LIBRARY}_{float(mean_str)}_{float(var_str)}"
)
if DATASET_PARAMS["multivariate"]:
    ROOT_FOLDER += "_multivariate"

# run for the first time to get all the means and stuff
training_params = run_gas_experiment(
    DATASET_NAME,
    DATASET_TYPE,
    DATASET_PARAMS,
    ROOT_FOLDER,
    NORMALIZER_NAME,
    NORMALIZER_INITIAL_GUESSES,
    NORMALIZER_BOUNDS,
    MEAN_LAYER_NAME,
    DL_MODEL_LIBRARY,
    DL_MODEL_NAME,
    DATASET_FILE_FOLDER,
    NORMALIZER_PARAMS,
    MEAN_LAYER_PARAMS,
    DL_MODEL_PARAMS,
    N_TRAINING_SAMPLES,
    N_TEST_SAMPLES,
    probabilistic=True,
)

# get the parameters needed to run the model part

n_features, context_length, prediction_length, freq, dataset, mean_layer, dl_model_name, dl_model_params, folders = training_params
n_train, n_test = dataset

Warming up train dataset...


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 112 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of 107 | elapsed:    2.4s remaining:   49.5s
[Parallel(n_jobs=-1)]: Done  16 out of 107 | elapsed:    2.6s remaining:   14.8s
[Parallel(n_jobs=-1)]: Done  27 out of 107 | elapsed:    2.6s remaining:    7.8s
[Parallel(n_jobs=-1)]: Done  38 out of 107 | elapsed:    2.7s remaining:    4.9s
[Parallel(n_jobs=-1)]: Done  49 out of 107 | elapsed:    2.7s remaining:    3.2s
[Parallel(n_jobs=-1)]: Done  60 out of 107 | elapsed:    2.8s remaining:    2.2s
[Parallel(n_jobs=-1)]: Done  71 out of 107 | elapsed:    2.8s remaining:    1.4s
[Parallel(n_jobs=-1)]: Done  82 out of 107 | elapsed:    2.8s remaining:    0.9s
[Parallel(n_jobs=-1)]: Done  93 out of 107 | elapsed:    2.8s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done 104 out of 107 | elapsed:    2.8s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done 107 out of 107 | elapsed:    2.9s finished
[Parallel(n_jobs=-1)]: Using backend Loky

Warming up test dataset...


[Parallel(n_jobs=-1)]: Done   5 out of 107 | elapsed:    0.7s remaining:   13.3s
[Parallel(n_jobs=-1)]: Done  16 out of 107 | elapsed:    0.7s remaining:    4.2s
[Parallel(n_jobs=-1)]: Done  27 out of 107 | elapsed:    0.8s remaining:    2.2s
[Parallel(n_jobs=-1)]: Done  38 out of 107 | elapsed:    0.8s remaining:    1.4s
[Parallel(n_jobs=-1)]: Done  49 out of 107 | elapsed:    0.8s remaining:    0.9s
[Parallel(n_jobs=-1)]: Done  60 out of 107 | elapsed:    0.8s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done  71 out of 107 | elapsed:    0.8s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done  82 out of 107 | elapsed:    0.8s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  93 out of 107 | elapsed:    0.9s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done 104 out of 107 | elapsed:    0.9s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 107 out of 107 | elapsed:    0.9s finished


Done.
Normalizing train dataset...
Saving normalizer parameters...
Saving normalized train dataset, means and vars...
Done.
Normalizing test dataset...
Done.
Saving normalized test dataset, means and vars...
Initializing the mean linear layer...
Initializing the estimator...
Training the estimator...


[09:06:36] /work/mxnet/src/base.cc:79: cuDNN lib mismatch: linked-against version 8101 != compiled-against version 8500.  Set MXNET_CUDNN_LIB_CHECKING=0 to quiet this warning.
100%|██████████| 100/100 [00:04<00:00, 23.55it/s, epoch=1/1, avg_epoch_loss=3.61]

Done.
Evaluating the estimator...
Done.


In [9]:
from data_manager import GluonTSDataManager

data_manager = GluonTSDataManager(DATASET_NAME, multivariate, DATASET_FILE_FOLDER, False)
data_manager = data_manager
n_features = data_manager.n_features
context_length = data_manager.context_length
prediction_length = data_manager.prediction_length
nn_train = data_manager.train_dataset
nn_test = data_manager.test_dataset
freq = data_manager.freq

n_train_sub = []
for ts in n_train:
    n_train_sub.append(
        {
            "start": ts["start"],
            "target": ts["target"],
            # "feat_dynamic_real": np.zeros_like(ts["feat_dynamic_real"]),
            # "feat_static_real": np.zeros_like(ts["feat_static_real"]),
        }
    )
n_test_sub = []
for ts in n_test:
    n_test_sub.append(
        {
            "start": ts["start"],
            "target": ts["target"],
            # "feat_dynamic_real": np.zeros_like(ts["feat_dynamic_real"]),
            # "feat_static_real": np.zeros_like(ts["feat_static_real"]),
        }
    )

In [2]:
import numpy as np
import mxnet

def are_dicts_equal(dict1, dict2):
    if dict1.keys() != dict2.keys():
        print(dict1.keys(), dict2.keys())
        return False

    for key in dict1.keys():
        if (isinstance(dict1[key], np.ndarray) and isinstance(dict2[key], np.ndarray)) or (isinstance(dict1[key], mxnet.ndarray.ndarray.NDArray) and isinstance(dict2[key], mxnet.ndarray.ndarray.NDArray)):
            if not np.array_equal(dict1[key], dict2[key]):
                print(key)
                return False
        elif dict1[key] != dict2[key]:
            print(key)
            return False

    return True

def are_lists_equal(list1, list2):
    if len(list1) != len(list2):
        print(len(list1), len(list2))
        return False
    i = 0
    for dict1, dict2 in zip(list1, list2):
        if not are_dicts_equal(dict1, dict2):
            print(i)
            return False
        i += 1

    return True

# are_lists_equal(n_train_sub, nn_train)

In [3]:
# load pickle file called "params.pickle"
with open('params.pickle', "rb") as f:
    n_params = pickle.load(f)

with open('params_default.pickle', "rb") as f:
    nn_params = pickle.load(f)


In [15]:
are_dicts_equal(n_params[-1], nn_params[-1])

True

In [10]:
# print the lens
print(len(n_params))
print(len(nn_params))

# print the keys of both dictionaries
print(n_params[0].keys())
print(nn_params[0].keys())

# limit n_params to only the ones that are in nn_params
# n_params = {k: v for k, v in n_params.items() if k in nn_params} # for direct dict
n_params = [{k: v for k, v in params.items() if k in nn_params[0]} for params in n_params] # for list of dicts

# print the keys of both dictionaries
print(n_params[0].keys())
print(nn_params[0].keys())

102
102
dict_keys(['feat_static_cat', 'past_time_feat', 'past_target', 'past_observed_values', 'future_time_feat', 'future_target', 'future_observed_values', 'past_feat_dynamic_real', 'feat_static_real'])
dict_keys(['feat_static_cat', 'past_time_feat', 'past_target', 'past_observed_values', 'future_time_feat', 'future_target', 'future_observed_values'])
dict_keys(['feat_static_cat', 'past_time_feat', 'past_target', 'past_observed_values', 'future_time_feat', 'future_target', 'future_observed_values'])
dict_keys(['feat_static_cat', 'past_time_feat', 'past_target', 'past_observed_values', 'future_time_feat', 'future_target', 'future_observed_values'])


In [7]:
are_lists_equal(n_params, nn_params)

/opt/conda/envs/clts2/lib/python3.10/site-packages/numpy/core/numeric.py:2457: VisibleDeprecationWarning: Creating an ndarray from nested sequences exceeding the maximum number of dimensions of 32 is deprecated. If you mean to do this, you must specify 'dtype=object' when creating the ndarray.
  a1, a2 = asarray(a1), asarray(a2)


past_time_feat
0


False

In [8]:
for i in range(len(n_params)):
    if not are_dicts_equal(n_params[i], nn_params[i]):
        print(i)

past_time_feat
0
past_time_feat
1
past_time_feat
2


In [54]:
from collections import defaultdict
import numpy as np

def combine(params):
    # Assuming n_params is your list of dictionaries
    combined = defaultdict(list)

    for d in params:
        for key, value in d.items():
            combined[key].append(value.asnumpy())

    # Convert lists of arrays to single arrays
    for key, value in combined.items():
        combined[key] = np.concatenate(value, axis=0)
        combined[key]=combined[key].squeeze().reshape(1,-1)
        # sort the values in the key
        combined[key] = np.sort(combined[key])
    return combined

In [58]:
n_params_combined = combine(n_params)
# save the combined params
with open('n_params_combined.pickle', "wb") as f:
    pickle.dump(n_params_combined, f)
nn_params_combined = combine(nn_params)
# save the combined params
with open('nn_params_default_combined.pickle', "wb") as f:
    pickle.dump(nn_params_combined, f)

In [60]:
# print the lens
print(len(n_params_combined))
print(len(nn_params_combined))

# print the keys of both dictionaries
print(n_params_combined.keys())
print(nn_params_combined.keys())

# limit n_params to only the ones that are in nn_params
# n_params_combined = {k: v for k, v in n_params_combined.items() if k in nn_params_combined} # for direct dict
# n_params_combined = [{k: v for k, v in params.items() if k in nn_params_combined[0]} for params in n_params_combined] # for list of dicts

# print the keys of both dictionaries
print(n_params_combined.keys())
print(nn_params_combined.keys())

7
7
dict_keys(['feat_static_cat', 'past_time_feat', 'past_target', 'past_observed_values', 'future_time_feat', 'future_target', 'future_observed_values'])
dict_keys(['feat_static_cat', 'past_time_feat', 'past_target', 'past_observed_values', 'future_time_feat', 'future_target', 'future_observed_values'])
dict_keys(['feat_static_cat', 'past_time_feat', 'past_target', 'past_observed_values', 'future_time_feat', 'future_target', 'future_observed_values'])
dict_keys(['feat_static_cat', 'past_time_feat', 'past_target', 'past_observed_values', 'future_time_feat', 'future_target', 'future_observed_values'])


In [61]:
are_dicts_equal(n_params_combined, nn_params_combined)

past_time_feat


False

In [82]:
import numpy as np

# Assuming array1 and array2 are your numpy arrays
for key in n_params_combined.keys():
    array1 = n_params_combined[key].squeeze()
    array2 = nn_params_combined[key].squeeze()
    diff_indices = np.where(array1 != array2)

    if diff_indices[0].size:
        print(f"{key} Arrays are not equal. {len(diff_indices[0])} out of {len(array1)} elements are different.")
        print("Differences at indices:", diff_indices)
        print("array1 values:", array1[diff_indices])
        print("array2 values:", array2[diff_indices])
    else:
        print("Arrays are equal.")


Arrays are equal.
past_time_feat Arrays are not equal. 173519 out of 336232 elements are different.
Differences at indices: (array([ 13620,  13621,  13622, ..., 336214, 336215, 336216]),)
array1 values: [-0.5       -0.5       -0.5       ...  2.8407333  2.8407333  2.8407333]
array2 values: [-0.4090909 -0.4090909 -0.4090909 ...  2.8401062  2.8401062  2.8401062]
past_target Arrays are not equal. 161617 out of 168116 elements are different.
Differences at indices: (array([  5097,   5098,   5099, ..., 168098, 168099, 168100]),)
array1 values: [0.9757815 0.9757815 0.9757815 ... 6.202237  6.202237  6.202237 ]
array2 values: [0.       0.       0.       ... 6.196274 6.196274 6.196274]
past_observed_values Arrays are not equal. 1266 out of 168116 elements are different.
Differences at indices: (array([5097, 5098, 5099, ..., 6360, 6361, 6362]),)
array1 values: [1. 1. 1. ... 1. 1. 1.]
array2 values: [0. 0. 0. ... 0. 0. 0.]
future_time_feat Arrays are not equal. 37364 out of 77592 elements are diff

# Checking mean vars

In [ ]:
import pickle
import pandas as pd
import numpy as np
%cd /mnt


/mnt


In [34]:
with open('train_0.pkl', 'rb') as f:
    train_0 = pickle.load(f).dataset
with open('train_1.pkl', 'rb') as f:
    train_1 = pickle.load(f).dataset
with open('train_2.pkl', 'rb') as f:
    train_2 = pickle.load(f).dataset
with open('train_3.pkl', 'rb') as f:
    train_3 = pickle.load(f).dataset
with open('train_4.pkl', 'rb') as f:
    train_4 = pickle.load(f).dataset

In [40]:
print(train_0[0]['means_vars'],'\n')
print(train_1[0]['means_vars'],'\n')
print(train_2[0]['means_vars'],'\n')
print(train_3[0]['means_vars'],'\n')

[[9.9999911e-01 9.9999809e-01 9.9999708e-01 ... 9.9927664e-01
  9.9927562e-01 9.9927461e-01]
 [1.8566617e-04 1.8668659e-04 1.8770700e-04 ... 9.1297494e-04
  9.1399468e-04 9.1501430e-04]] 

[[9.9999911e-01 9.9999809e-01 9.9999708e-01 ... 9.9927664e-01
  9.9927562e-01 9.9927461e-01]
 [1.8566617e-04 1.8668659e-04 1.8770700e-04 ... 9.1297494e-04
  9.1399468e-04 9.1501430e-04]] 

[[9.9999911e-01 9.9999809e-01 9.9999708e-01 ... 9.9927664e-01
  9.9927562e-01 9.9927461e-01]
 [1.8566617e-04 1.8668659e-04 1.8770700e-04 ... 9.1297494e-04
  9.1399468e-04 9.1501430e-04]] 

[[9.9999911e-01 9.9999809e-01 9.9999708e-01 ... 9.9927664e-01
  9.9927562e-01 9.9927461e-01]
 [1.8566617e-04 1.8668659e-04 1.8770700e-04 ... 9.1297494e-04
  9.1399468e-04 9.1501430e-04]] 



In [37]:
# iterate through train_0 and train_1 and compare the means_vars
array1 = train_0
array2 = train_4 
for i in range(len(array1)):
    if not np.array_equal(array1[i]['means_vars'], array2[i]['means_vars']):
        print('means_vars',i)
        print(array1[i]['means_vars'])
        print(array2[i]['means_vars'])
    if not np.array_equal(array1[i]['target'], array2[i]['target']):
        print('target',i)
        print(array1[i]['target'])
        print(array2[i]['target'])
    if not np.array_equal(array1[i]['start'], array2[i]['start']):
        print('start',i)
        print(array1[i]['start'])
        print(array2[i]['start'])
    if not np.array_equal(array1[i]['gas_params'], array2[i]['gas_params']):
        print('gas_params', i)
        print(array1[i]['gas_params'])
        print(array2[i]['gas_params'])

In [41]:
from data_manager import GluonTSDataManager

data_manager = GluonTSDataManager('fred_md', False, None, False)
data_manager = data_manager
n_features = data_manager.n_features
context_length = data_manager.context_length
prediction_length = data_manager.prediction_length
nn_train = data_manager.train_dataset
nn_test = data_manager.test_dataset
freq = data_manager.freq

In [45]:
data_manager.train_dataset[0]['target'].std()

1.6153463